In [10]:
import pandas as pd


# Import CSV of fitbit weartime using heartrates
df = pd.read_csv('fitbitWearTimeViaHR_merged.csv')

Handle the weartime data: return Ids, average wear time via heartrate data, number of days worn, and largest strand of consecutive days worn.

In [11]:
df['Day'] = pd.to_datetime(df['Day'], format='%m/%d/%Y', errors='coerce')
df

,Id,Day,TotalMinutesWearTime,PercentageWearTime
0,10001,2024-10-24,0,0.000000
1,10001,2024-10-25,0,0.000000
2,10001,2024-10-26,0,0.000000
3,10001,2024-10-27,0,0.000000
4,10001,2024-10-28,0,0.000000
...,...,...,...,...
4335,56214,2024-10-26,0,0.000000
4336,56214,2024-10-27,10,0.694444
4337,56214,2024-10-28,3,0.208333
4338,56214,2024-10-29,2,0.138889


In [12]:
# Return wear time and sync data stats!

# FIND CONSECUTIVE DAYS
def consecutive_days(group, minsGroup = None):
    conDays = 0
    currentStreak = 0
    previousDay = None

    for i, dateTime in enumerate(group):
        currentDay = dateTime
        if(previousDay is None or abs((currentDay - previousDay).days) == 1):
            if(minsGroup is not None):
                if(minsGroup.iloc[i] > 0):
                    currentStreak += 1
                else:
                    currentStreak = 0
            else:
                currentStreak += 1
            if(currentStreak > conDays):
                conDays = currentStreak
        else :
            if(abs((currentDay - previousDay).days) != 0):
                currentStreak = 1
        previousDay = currentDay
    return conDays
    

# We will keep track of average documented weartime, total number of days worn, number of syncs, and last time synced.
reminder_wearDf = df.groupby('Id').apply(lambda x: pd.Series({
    'AverageWearTime': x['PercentageWearTime'].mean(),
    'DaysWorn': (x['TotalMinutesWearTime'] > 0).sum(),
    'ConsecutiveDaysWorn': consecutive_days(x['Day'], x['TotalMinutesWearTime'])
})).reset_index()
reminder_wearDf

,Id,AverageWearTime,DaysWorn,ConsecutiveDaysWorn
0,10001,0.000000,0.0,0.0
1,10101,0.000000,0.0,0.0
2,10102,0.000000,0.0,0.0
3,10103,0.000000,0.0,0.0
4,10104,0.000000,0.0,0.0
...,...,...,...,...
615,56211,87.867063,7.0,7.0
616,56212,30.575397,4.0,4.0
617,56213,45.089286,6.0,4.0
618,56214,0.466270,5.0,3.0


Handle the sync data: return Ids, last day synced, number of days synced, and largest strand of consecutive days synced.

In [13]:
# Read sync data
syncDf = pd.read_csv('syncEvents_merged.csv')
syncDf['DateTime'] = pd.to_datetime(syncDf['DateTime'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce').dt.date


syncDf

,Id,DateTime,SyncDateUTC,Provider,DeviceName
0,51103,2024-10-24,10/24/2024 5:10:13 AM,Fitbit,Inspire 3
1,51103,2024-10-24,10/24/2024 5:25:20 AM,Fitbit,Inspire 3
2,51103,2024-10-24,10/24/2024 5:40:26 AM,Fitbit,Inspire 3
3,51103,2024-10-24,10/24/2024 5:55:32 AM,Fitbit,Inspire 3
4,51103,2024-10-24,10/24/2024 6:10:38 AM,Fitbit,Inspire 3
...,...,...,...,...,...
31231,56214,2024-10-29,10/29/2024 3:23:16 PM,Fitbit,Inspire 3
31232,56214,2024-10-29,10/29/2024 3:38:22 PM,Fitbit,Inspire 3
31233,56214,2024-10-29,10/30/2024 4:34:03 AM,Fitbit,Inspire 3
31234,56214,2024-10-29,10/30/2024 4:34:08 AM,Fitbit,Inspire 3


In [14]:
# Function to determine days synced
def uniqueDays(group):
    days = 0
    previousDay = None

    for dateTime in group:
        currentDay = str(dateTime)
        if(previousDay is None or currentDay[8:10] != previousDay[8:10]):
            days += 1
        previousDay = currentDay
    return days


# Sort last day synced and total number of syncs
reminder_syncDf = syncDf.groupby('Id').agg(
    LastDaySynced = ('DateTime', 'last'),
    DaysSynced = ('DateTime', uniqueDays),
    ConsecutiveDaysSynced = ('DateTime', consecutive_days)
).reset_index()

reminder_syncDf

,Id,LastDaySynced,DaysSynced,ConsecutiveDaysSynced
0,51103,2024-10-30,7,7
1,51105,2024-10-30,3,3
2,51106,2024-10-26,1,1
3,51114,2024-10-30,7,7
4,51115,2024-10-29,6,6
...,...,...,...,...
87,56211,2024-10-30,5,3
88,56212,2024-10-28,3,2
89,56213,2024-10-30,4,2
90,56214,2024-10-29,4,3


Combine weartime and sync dataframes! (NaN may appear where data is missing.)

In [15]:
# Combine Weartime and Sync dataframes
reminder_df = pd.merge(reminder_wearDf, reminder_syncDf, on = 'Id', how = 'outer')
reminder_df.fillna(0, inplace=True)
reminder_df

,Id,AverageWearTime,DaysWorn,ConsecutiveDaysWorn,LastDaySynced,DaysSynced,ConsecutiveDaysSynced
0,10001,0.000000,0.0,0.0,0,0.0,0.0
1,10101,0.000000,0.0,0.0,0,0.0,0.0
2,10102,0.000000,0.0,0.0,0,0.0,0.0
3,10103,0.000000,0.0,0.0,0,0.0,0.0
4,10104,0.000000,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...
615,56211,87.867063,7.0,7.0,2024-10-30,5.0,3.0
616,56212,30.575397,4.0,4.0,2024-10-28,3.0,2.0
617,56213,45.089286,6.0,4.0,2024-10-30,4.0,2.0
618,56214,0.466270,5.0,3.0,2024-10-29,4.0,3.0


Ignore personalized messages for now:

In [7]:
# Determine what messages people should receive?
# Set threshold values for consistency
# min_consecutive_days_worn = 3
# min_average_wear_time = 50.0
# max_days_not_synced = 3

# # Generate personalized reminders
# reminder_df['Reminder'] = ''

# for index, row in reminder_df.iterrows():
#     reminder = ''
    
#     if row['ConsecutiveDaysWorn'] < min_consecutive_days_worn:
#         reminder += f"You have only worn the device for {row['ConsecutiveDaysWorn']} consecutive days in the last month. Please aim to wear it for at least {min_consecutive_days_worn} consecutive days! "
    
#     if row['AverageWearTime'] < min_average_wear_time:
#         reminder += f"In the last month, your average wear time was {row['AverageWearTime']:.2f}%. Next month, please remember to wear it more consistently to further help our reseach. Let's aim for {min_consecutive_days_worn} consecutive days!"
    
#     last_sync_date = pd.to_datetime(row['LastDaySynced'])
#     days_since_last_sync = (pd.Timestamp.now() - last_sync_date).days
    
#     if days_since_last_sync > max_days_not_synced:
#         reminder += f"It has been {days_since_last_sync} days since your last sync. Please remember to sync your device regularly. "
    
#     if row['ConsecutiveDaysSynced'] < min_consecutive_days_worn:
#         reminder += f"You have only synced the device for {row['ConsecutiveDaysSynced']} consecutive days. Please aim to sync it for at least {min_consecutive_days_worn} consecutive days. "
    
#     if reminder == '':
#         reminder = "Great job! You have been wearing and syncing your device consistently. Keep up the good work!"
    
#     reminder_df.at[index, 'Reminder'] = reminder.strip()

# reminder_df

Return a new CSV file, reminder_ids.csv, with all of the data!

In [16]:
# Save the reminders to a CSV file
reminder_df.to_csv('reminder_ids.csv', index=False)